Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [2]:
import pip

def install(package):
    pip.main(['install', package])

# Example
# if __name__ == '__main__':
    # install('tensorflow')

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.
Load the data
--

In [6]:
# file for home computer
# pickle_file = '/Users/gundeep/Documents/Notebooks/deep-learning/notMNIST.pickle'

# file for office computer
pickle_file = '/Users/Gundeep/Documents/Notebooks/deep-learning/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

Just for testing different kind of array operations
---

In [7]:
test_a = np.arange(10)
print(test_a)
test_b = test_a[:,None]
print('kjust')
print(test_b)
print(test_b.shape)
print('testngsome more shitn')
test_ya = np.array([1,2,3,4])
test_ya1 = test_ya[:,None]
test_ya2 = test_ya[None,:]
print(test_ya2)
test_yo = np.array([[1,2,3,4],[1,2,3,4]])
test_yoyo = np.array([[1,1],[2,2],[3,3],[4,4]])
print(test_yoyo)
test_yoyo = test_yoyo[None,:]
print(test_yoyo)
# print(test_yo)
print(test_ya.shape)
print(test_ya1.shape)
print(test_ya2.shape)
print(test_yo.shape)
print(test_yoyo.shape)
# test_c = test_a == test_b
# print(test_c)
# test_d = test_b == test_a
# print(test_d)

[0 1 2 3 4 5 6 7 8 9]
kjust
[[0]
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]
 [7]
 [8]
 [9]]
(10, 1)
testngsome more shitn
[[1 2 3 4]]
[[1 1]
 [2 2]
 [3 3]
 [4 4]]
[[[1 1]
  [2 2]
  [3 3]
  [4 4]]]
(4,)
(4, 1)
(1, 4)
(2, 4)
(1, 4, 2)


Reformat the data to 1d array
---

In [8]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


Tf zeros testing
---

In [9]:
zr = tf.zeros(6)
zr2 = tf.zeros([6])
print(zr2)
print(zr)
tf.reset_default_graph()

Tensor("zeros_1:0", shape=(6,), dtype=float32)
Tensor("zeros:0", shape=(6,), dtype=float32)


How TF works
---
We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [24]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  logits_first_10 = logits[:1]
  cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
  print('shape of cross entropy = ' + str(cross_entropy.get_shape()))
  loss = tf.reduce_mean(cross_entropy)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

  print('shape of train prediction - ' + str(train_prediction.get_shape()))

shape of cross entropy = (10000,)
shape of train prediction - (10000, 10)


Let's run this computation and iterate:

In [27]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions, logits_10 = session.run([optimizer, loss, train_prediction, logits_first_10])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('logits_10 = %s'%str(logits_10))
      print('precdiction 1 = %s'%str(predictions[0]))
      print('precdiction sum 1 = %s'%str(np.sum(predictions[0],0)))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 18.470297
logits_10 = [[-16.30040359   1.78290987   6.35871887  -1.18855476   2.47864747
  -12.34409332   0.14851618 -14.62033081  15.91209126  -2.37953568]]
precdiction 1 = [  1.02390537e-14   7.30706347e-07   7.09563465e-05   3.74327307e-08
   1.46520222e-06   5.35135196e-13   1.42539335e-07   5.49422128e-14
   9.99926686e-01   1.13766907e-08]
precdiction sum 1 = 1.0
Training accuracy: 12.0%
Validation accuracy: 14.6%
Loss at step 100: 2.416394
logits_10 = [[-15.40691853   6.52225304   7.17355585  -4.59951591   4.75465202
  -13.8651104    6.31645298  -8.58014297   8.6554327  -11.12266731]]
precdiction 1 = [  2.42554778e-11   8.10080245e-02   1.55376464e-01   1.19785750e-06
   1.38314804e-02   1.13346728e-10   6.59401864e-02   2.23686811e-08
   6.83842659e-01   1.75969295e-09]
precdiction sum 1 = 1.0
Training accuracy: 70.9%
Validation accuracy: 70.1%
Loss at step 200: 1.908222
logits_10 = [[-14.04288769   6.03747749   7.54892206  -6.16764116   4.35214806
 

Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [22]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
  loss = tf.reduce_mean(cross_entropy)
  print('shape of cross entropy = ' + str(cross_entropy.get_shape()))
  print('shape of loss = ' + str(loss.get_shape()))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

shape of cross entropy = (128,)
shape of loss = ()


Let's run it:

In [15]:
print(train_labels.shape[0])

200000


In [18]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 19.872808
Minibatch accuracy: 4.7%
Validation accuracy: 9.1%
Minibatch loss at step 500: 1.181864
Minibatch accuracy: 79.7%
Validation accuracy: 75.2%
Minibatch loss at step 1000: 1.404499
Minibatch accuracy: 75.0%
Validation accuracy: 76.7%
Minibatch loss at step 1500: 0.753638
Minibatch accuracy: 81.2%
Validation accuracy: 77.5%
Minibatch loss at step 2000: 0.906474
Minibatch accuracy: 82.8%
Validation accuracy: 77.5%
Minibatch loss at step 2500: 0.978004
Minibatch accuracy: 75.0%
Validation accuracy: 78.1%
Minibatch loss at step 3000: 0.937416
Minibatch accuracy: 82.0%
Validation accuracy: 79.3%
Test accuracy: 86.7%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

Solution 1 : Adding Relu to the weights
---

In [19]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.truncated_normal([image_size * image_size, num_labels])
  # add a relu to the weights
  relufied_weights = tf.Variable(tf.nn.relu(weights))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, relufied_weights) + biases
  cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
  loss = tf.reduce_mean(cross_entropy)
  print('shape of cross entropy = ' + str(cross_entropy.get_shape()))
  print('shape of loss = ' + str(loss.get_shape()))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, relufied_weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, relufied_weights) + biases)

shape of cross entropy = (128,)
shape of loss = ()


In [44]:
arr = np.arange(15).reshape(3,5)
print(arr)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]]


Test run with relu

In [27]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

# print('shape of weights = ' + str(relufied_weights.get_shape()))
# sess = tf.InteractiveSession()
# for index in np.arange(10):
    # print('weights[%d]:' % (index) )
    # print(relufied_weights[index])

Initialized
Minibatch loss at step 0: 10.728715
Minibatch accuracy: 4.7%
Validation accuracy: 12.6%
Minibatch loss at step 500: 0.684151
Minibatch accuracy: 83.6%
Validation accuracy: 77.6%
Minibatch loss at step 1000: 0.819420
Minibatch accuracy: 79.7%
Validation accuracy: 78.5%
Minibatch loss at step 1500: 0.497022
Minibatch accuracy: 85.9%
Validation accuracy: 79.2%
Minibatch loss at step 2000: 0.663290
Minibatch accuracy: 85.9%
Validation accuracy: 79.6%
Minibatch loss at step 2500: 0.697769
Minibatch accuracy: 80.5%
Validation accuracy: 80.6%
Minibatch loss at step 3000: 0.756932
Minibatch accuracy: 81.2%
Validation accuracy: 80.7%
Test accuracy: 88.3%
shape of weights = (784, 10)
weights[0]:
Tensor("strided_slice_13:0", shape=(10,), dtype=float32)
weights[1]:
Tensor("strided_slice_14:0", shape=(10,), dtype=float32)
weights[2]:
Tensor("strided_slice_15:0", shape=(10,), dtype=float32)
weights[3]:
Tensor("strided_slice_16:0", shape=(10,), dtype=float32)
weights[4]:
Tensor("strided_s

Solution 2 : w2 x Relu(image x w1+bias1) + bias2
---

In [45]:
batch_size = 128
num_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
  weights2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
  biases1 = tf.Variable(tf.zeros([num_nodes]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  # add a relu to the output after the first matrix multiplication
  relufied_input = tf.nn.relu(tf.matmul(tf_train_dataset,weights1) + biases1)
  
  
  # Training computation.
  logits = tf.matmul(relufied_input, weights2) + biases2
  cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
  loss = tf.reduce_mean(cross_entropy)
  print('shape of cross entropy = ' + str(cross_entropy.get_shape()))
  print('shape of loss = ' + str(loss.get_shape()))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

shape of cross entropy = (128,)
shape of loss = ()


In [46]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

# print('shape of weights = ' + str(relufied_weights.get_shape()))
# sess = tf.InteractiveSession()
# for index in np.arange(10):
    # print('weights[%d]:' % (index) )
    # print(relufied_weights[index])

Initialized
Minibatch loss at step 0: 309.046570
Minibatch accuracy: 15.6%
Validation accuracy: 27.3%
Minibatch loss at step 500: 13.921703
Minibatch accuracy: 82.0%
Validation accuracy: 81.0%
Minibatch loss at step 1000: 7.931565
Minibatch accuracy: 78.9%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 6.068065
Minibatch accuracy: 89.1%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 2.987371
Minibatch accuracy: 83.6%
Validation accuracy: 81.7%
Minibatch loss at step 2500: 3.109516
Minibatch accuracy: 82.0%
Validation accuracy: 82.0%
Minibatch loss at step 3000: 1.797323
Minibatch accuracy: 87.5%
Validation accuracy: 82.2%
Test accuracy: 89.5%
